<img src="https://www3.um.edu.uy/logoum.jpg" width=300>
<h1 align="center">Thesis - IMF MRR</h1> 
<h2 align="center">Alejo Paullier</h2> 

https://lkpy.lenskit.org/en/stable/knn.html

In [1]:
from lenskit import batch, topn, util
from lenskit import crossfold as xf
from lenskit.algorithms import Recommender, als
from lenskit.algorithms.als import ImplicitMF
from lenskit import topn
import pandas as pd
import numpy as np

ratings = pd.read_csv('D:\\Escritorio\\UM\\Tesis\\ML 1M\\ratings.dat', sep='::',engine='python',
                      names=['user', 'item', 'rating', 'timestamp'])

def eval(aname, algo, train, test):
    fittable = util.clone(algo) # Object cloning (create an exact copy of the original object)
    fittable = Recommender.adapt(fittable) 
    fittable.fit(train) # Entrenamos el algoritmo con el training dataset
    users = test.user.unique() # Devuelve un array con los users unicos en el testing dataset
    # now we run the recommender
    recs = batch.recommend(fittable, users, 100)
    # add the algorithm name for analyzability
    recs['Algorithm'] = aname
    return recs

In [2]:
def model_trainer(features,reg,weight):
    all_recs = []
    test_data = []
    
    features=int(features)
    imf = ImplicitMF(features=features, iterations=100, reg=reg, weight=weight) # define algorithm# define algorithm
    
    for train, test in xf.partition_users(ratings[['user', 'item', 'rating']], 5, xf.SampleFrac(0.2)):
        test_data.append(test) # save testing data
        all_recs.append(eval('IMF', imf, train, test))
    
    all_recs = pd.concat(all_recs, ignore_index=True)
    test_data = pd.concat(test_data, ignore_index=True)
    rla = topn.RecListAnalysis()
    rla.add_metric(topn.recip_rank)
    results = rla.compute(all_recs, test_data)
                                                    
    return results["recip_rank"].mean()

In [3]:
from hyperopt import fmin, tpe, hp, STATUS_OK
array=[]
def objective(params):
    features = params['features']
    reg = params['reg']
    weight=params['weight']
    metric = model_trainer(features,reg,weight)
    array.append([features,reg,weight,metric])
    return {'loss': -metric, 'status': STATUS_OK}

space={'features': hp.uniform('features', 1, 100),
       'reg': hp.uniform('reg',0.01,0.4),
      'weight': hp.uniform('weight',0,100)}

best = fmin(objective, space, algo=tpe.suggest,max_evals=100)

  1%|▍                                            | 1/100 [11:19<18:41:45, 679.85s/it, best loss: -0.42010118405242325]


KeyboardInterrupt: 

In [4]:
array

[[88.13313334581372,
  0.2083226698471299,
  13.738652763733272,
  0.42010118405242325]]

In [6]:
metric = pd.DataFrame(array)
metric.columns = ['features','reg','weight','MRR']
metric.head(5)

,features,reg,weight,MRR
0,56.659927,0.223499,54.759794,0.254021
1,35.151787,0.246733,19.429494,0.377435
2,35.207998,0.019786,12.307978,0.447460
3,17.405235,0.307952,7.187427,0.514315
4,28.183507,0.130176,74.600217,0.197891


In [4]:
array # 504.81

[[5.808917972506118,
  0.17690643550654947,
  71.0299435288029,
  0.2689309256827014],
 [93.28747623691821,
  0.10890038765467326,
  6.563721403043421,
  0.5079646042789487],
 [14.624720563828923,
  0.07043509687153773,
  89.37045295307655,
  0.18603701116412955],
 [17.08085817646334,
  0.23106354566687448,
  95.69987044577353,
  0.1710784679215075],
 [5.078053914418048, 0.1335947519436751, 46.92315820663576, 0.30972650810361],
 [83.05393599813333,
  0.22283058949443696,
  71.10932415670769,
  0.2569333929663447],
 [88.03563349046378,
  0.32554173146379234,
  0.03724996982303885,
  0.6620307192922975],
 [98.73788446709077,
  0.26039395851189107,
  52.1837380331143,
  0.3068503884962579],
 [54.29784071108095,
  0.3969988750655781,
  58.114985123111765,
  0.2469432762022818],
 [33.79876718257659,
  0.013047689037178626,
  40.66277010947892,
  0.27622816532396766],
 [16.28622773645938,
  0.24281177871573176,
  81.00746921339962,
  0.1896903593331915],
 [97.6856107161698,
  0.3544555724051

In [4]:
array #600.42

[[56.659927006218936,
  0.2234988513555401,
  54.75979371393459,
  0.25402068691769447],
 [35.151787479317235,
  0.24673311029575584,
  19.42949379454878,
  0.3774352747038264],
 [35.20799827370031,
  0.019785960417483393,
  12.307978118173002,
  0.4474596445416308],
 [17.405234794166144,
  0.3079521323044522,
  7.18742743906613,
  0.5143152846599908],
 [28.183507451234487,
  0.1301758540687238,
  74.60021712802113,
  0.19789062195526289],
 [30.938274677071547,
  0.2831465611636473,
  38.347694567090926,
  0.2872549092711304],
 [17.743106143989746,
  0.16422098494583418,
  74.91850496597091,
  0.20263616075592106],
 [85.36959871591239,
  0.21034239110814118,
  78.24760571236182,
  0.2548098112680389],
 [27.14571246153346,
  0.2599896553765361,
  53.60913321007358,
  0.2408209219816516],
 [86.18196789541571,
  0.30471374666422035,
  80.88512719799454,
  0.25281980607345356],
 [12.469582679314712,
  0.15210641070313896,
  15.302743299241728,
  0.4361217230009801],
 [46.441701674786884,
 

In [5]:
array = [[56.659927006218936,
  0.2234988513555401,
  54.75979371393459,
  0.25402068691769447],
 [35.151787479317235,
  0.24673311029575584,
  19.42949379454878,
  0.3774352747038264],
 [35.20799827370031,
  0.019785960417483393,
  12.307978118173002,
  0.4474596445416308],
 [17.405234794166144,
  0.3079521323044522,
  7.18742743906613,
  0.5143152846599908],
 [28.183507451234487,
  0.1301758540687238,
  74.60021712802113,
  0.19789062195526289],
 [30.938274677071547,
  0.2831465611636473,
  38.347694567090926,
  0.2872549092711304],
 [17.743106143989746,
  0.16422098494583418,
  74.91850496597091,
  0.20263616075592106],
 [85.36959871591239,
  0.21034239110814118,
  78.24760571236182,
  0.2548098112680389],
 [27.14571246153346,
  0.2599896553765361,
  53.60913321007358,
  0.2408209219816516],
 [86.18196789541571,
  0.30471374666422035,
  80.88512719799454,
  0.25281980607345356],
 [12.469582679314712,
  0.15210641070313896,
  15.302743299241728,
  0.4361217230009801],
 [46.441701674786884,
  0.27437670110731016,
  84.2743054855266,
  0.20065234566996104],
 [3.905517979627161,
  0.18755218862826928,
  93.94151672166208,
  0.2786788332269548],
 [24.91257450393174,
  0.2197880322796414,
  51.58969758616716,
  0.24178618190060278],
 [98.76204210194581,
  0.23381819652842503,
  52.65250266981444,
  0.3011389566623602],
 [82.09050519297196,
  0.35925946750986587,
  48.243286969758714,
  0.29012789072236295],
 [30.41741162317053,
  0.1467016194441069,
  74.39946908918022,
  0.19711479967915993],
 [81.98251328824375,
  0.31833489093222106,
  82.6431941291364,
  0.2522389952193834],
 [21.065914435791484,
  0.3869066637693373,
  14.568534847178805,
  0.43503834600864794],
 [49.00062731318486,
  0.20780230210851422,
  3.312922232400406,
  0.5924835689730238],
 [61.983825675134796,
  0.3424785518800455,
  3.0074038974851245,
  0.5924329324236549],
 [63.78788169085313,
  0.03856329033676237,
  0.48624146229221443,
  0.664431678186597],
 [67.61346490087416,
  0.09884789341747303,
  30.252212817696922,
  0.3237428673212509],
 [46.19375815941536,
  0.010442414388698775,
  0.56566946365245,
  0.6557429050336261],
 [69.95875218550962,
  0.011425301090525152,
  23.576268025912537,
  0.3503754817367249],
 [41.42744429560905,
  0.05533053462211035,
  0.5220449992995447,
  0.6584114160384916],
 [55.64983959379204,
  0.06251424488239224,
  31.68542172433416,
  0.31229893478457577],
 [40.72145127884045,
  0.05912872027145705,
  61.47943834867287,
  0.22488950598199078],
 [74.19720301593715,
  0.058016579077388435,
  37.874511021654364,
  0.30660391215871924],
 [56.21175304373295,
  0.08740513046511544,
  62.54676282999738,
  0.23825820969476108],
 [98.99133092190837,
  0.03288361868489178,
  24.291363051617623,
  0.36444132067061524],
 [39.3026330241985,
  0.11553337962532204,
  11.550655789094364,
  0.4590137871292283],
 [91.78817826084669,
  0.038165793539228166,
  7.967007647455617,
  0.47992582884332524],
 [61.851946426522375,
  0.08958071079243687,
  45.23082196353672,
  0.27027415461241766],
 [7.605052045934805,
  0.1756164208402795,
  18.79551475690287,
  0.40057785905379734],
 [73.87780645826348,
  0.12339483345095151,
  8.1834897934876,
  0.48398058460669796],
 [36.93503518764684,
  0.034733139391165284,
  99.76375947339889,
  0.18226248446751755],
 [51.176352306555486,
  0.07409281233129658,
  27.91525826302651,
  0.3290616988860671],
 [44.714617920915416,
  0.10899723591647649,
  38.4047562525377,
  0.2815247526280663],
 [62.817243757879815,
  0.13255914784743686,
  1.210931991656119,
  0.6369740565460031],
 [34.25896466692174,
  0.19204621050028794,
  20.43300251616891,
  0.382672168475204],
 [5.808917972506118,
  0.17690643550654947,
  71.0299435288029,
  0.2689309256827014],
 [93.28747623691821,
  0.10890038765467326,
  6.563721403043421,
  0.5079646042789487],
 [14.624720563828923,
  0.07043509687153773,
  89.37045295307655,
  0.18603701116412955],
 [17.08085817646334,
  0.23106354566687448,
  95.69987044577353,
  0.1710784679215075],
 [5.078053914418048, 0.1335947519436751, 46.92315820663576, 0.30972650810361],
 [83.05393599813333,
  0.22283058949443696,
  71.10932415670769,
  0.2569333929663447],
 [88.03563349046378,
  0.32554173146379234,
  0.03724996982303885,
  0.6620307192922975],
 [98.73788446709077,
  0.26039395851189107,
  52.1837380331143,
  0.3068503884962579],
 [54.29784071108095,
  0.3969988750655781,
  58.114985123111765,
  0.2469432762022818],
 [33.79876718257659,
  0.013047689037178626,
  40.66277010947892,
  0.27622816532396766],
 [16.28622773645938,
  0.24281177871573176,
  81.00746921339962,
  0.1896903593331915],
 [97.6856107161698,
  0.35445557240515113,
  17.796082538185022,
  0.3929436271102338],
 [30.066088602161006,
  0.2942691259074041,
  62.414363189358866,
  0.21223319250933792],
 [67.02580822464054,
  0.06977096382060011,
  83.9721840646907,
  0.23203958032136413],
 [53.19466879023198,
  0.05249138960680773,
  96.34039689117935,
  0.20667947285162866],
 [46.28866365927022,
  0.34457083846206776,
  64.63860727549918,
  0.2295382963002187],
 [30.20086542592027,
  0.0403060101793221,
  33.70051547601275,
  0.30049891109328436],
 [67.80652462761063,
  0.06472809507585485,
  49.32703291725689,
  0.27481803483539985],
 [38.81137792410345,
  0.10561351108022633,
  94.87722507835163,
  0.1818070897874788],
 [34.54761985544079,
  0.30927070396585643,
  57.77384839224994,
  0.23138207312707518],
 [86.31014158382995,
  0.17491678157889717,
  4.294794276904425,
  0.5512757304965442],
 [85.0438957191429,
  0.18645828357412156,
  1.3035659995576898,
  0.6356871340124012],
 [75.94442912346187,
  0.18463566924524902,
  18.306098237587875,
  0.37723920323951354],
 [64.71434628262506, 0.2836739063886472, 14.946729196455, 0.40641708493394996],
 [86.72431348687903,
  0.3989392024089734,
  30.230208364463095,
  0.3400428024950368],
 [77.88744366739664,
  0.3345040901162304,
  2.673158169441945,
  0.6009887868029231],
 [93.99716115653366,
  0.13476388675454798,
  27.168388361988406,
  0.3452795010256785],
 [75.25399605071762,
  0.20282141526786607,
  11.009276393286653,
  0.44107152527219856],
 [58.31551012609657,
  0.36915418257459093,
  24.342230969499212,
  0.33737478735673543],
 [89.86867708452857,
  0.14982112206532885,
  0.8952936752503606,
  0.6379519031605015],
 [92.60036406906293,
  0.15257234820900026,
  9.688217651084651,
  0.460839313130385],
 [61.54633483988172,
  0.09273616353097064,
  0.028865293668985548,
  0.6613944730552013],
 [46.06441925640232,
  0.1088156247298002,
  37.140443538845126,
  0.2928509520334906],
 [61.96145894368485,
  0.012250222584969633,
  22.716615509779793,
  0.36189422076782785],
 [71.82803816395692,
  0.26245757297220845,
  9.121020652422741,
  0.46695999945171457],
 [81.13312364755097,
  0.09409932710968914,
  41.49061569844233,
  0.3049730487027617],
 [24.396845272208935,
  0.3217622183507526,
  74.85491618573383,
  0.19035895043616538],
 [45.76950132257882,
  0.22342528940999387,
  14.750801626470446,
  0.4123942539103378],
 [2.657642294950662,
  0.03247884476691801,
  0.23857172981555796,
  0.43256902259433344],
 [59.01447978830319,
  0.09189198905507531,
  43.86851891680561,
  0.2788239855918001],
 [10.826199855815396,
  0.3822553842167539,
  20.127184531127053,
  0.4003853461672648],
 [40.36081729431094,
  0.2855694423843841,
  6.125859941499894,
  0.5397163445872448],
 [97.91173421919154,
  0.25139424972830904,
  31.257041620507643,
  0.3430936400791403],
 [70.28423563305125,
  0.20178095314037847,
  13.760316991570859,
  0.4139637899378668],
 [22.967736480258466,
  0.1301057182861049,
  25.942964173660236,
  0.35184707405401383],
 [79.93398849674831,
  0.26968604437575805,
  54.39345026405129,
  0.2782429625769874],
 [52.25679428419094,
  0.166854153377572,
  35.61035348646122,
  0.29825859266112986],
 [57.04951647222507,
  0.08733612996265305,
  68.11413829968933,
  0.23075698554517549],
 [63.313460156422366,
  0.30631951078641606,
  76.80557560670006,
  0.23487975630086358],
 [72.01129810455961,
  0.23244767898816576,
  89.57480031896216,
  0.23104759521725393],
 [90.01718213467106,
  0.023791284947177416,
  20.88161064394815,
  0.37317708074430617],
 [49.49794491438451,
  0.3622468355773766,
  6.671881631949397,
  0.5169992424384212],
 [40.183533725225566,
  0.0639446333705389,
  50.83053119667665,
  0.24825678180287042],
 [67.95572761710206,
  0.12376726136095627,
  29.098742279248885,
  0.330019356106269],
 [83.09626610072253,
  0.38256112808099335,
  61.34642776396784,
  0.26804201628183205],
 [96.58067831721367,
  0.344854681395041,
  39.84962285328966,
  0.31545116834776904],
 [75.16693923602344,
  0.23877009743680555,
  46.10970177289386,
  0.28806794998736096],
 [54.661074126756766,
  0.15874041791239052,
  16.834417099083595,
  0.3918822896239884],
[88.13313334581372,
  0.2083226698471299,
  13.738652763733272,
  0.42010118405242325]]

In [7]:
metric.sort_values(by=['MRR'],ascending=False).head(10)

,features,reg,weight,MRR
21,63.787882,0.038563,0.486241,0.664432
47,88.035633,0.325542,0.037250,0.662031
72,61.546335,0.092736,0.028865,0.661394
25,41.427444,0.055331,0.522045,0.658411
23,46.193758,0.010442,0.565669,0.655743
70,89.868677,0.149821,0.895294,0.637952
39,62.817244,0.132559,1.210932,0.636974
62,85.043896,0.186458,1.303566,0.635687
66,77.887444,0.334504,2.673158,0.600989
19,49.000627,0.207802,3.312922,0.592484


In [8]:
metric.to_csv('metric_MRR.csv', index=False)

In [2]:
metric = pd.read_csv('C:\\Users\\Alejo\\Tesis\\1M\\Implicit Matrix Factorization\\results\\metric_MRR.csv')
metric

,features,reg,weight,MRR
0,56.659927,0.223499,54.759794,0.254021
1,35.151787,0.246733,19.429494,0.377435
2,35.207998,0.019786,12.307978,0.447460
3,17.405235,0.307952,7.187427,0.514315
4,28.183507,0.130176,74.600217,0.197891
...,...,...,...,...
95,83.096266,0.382561,61.346428,0.268042
96,96.580678,0.344855,39.849623,0.315451
97,75.166939,0.238770,46.109702,0.288068
98,54.661074,0.158740,16.834417,0.391882


In [4]:
%matplotlib qt
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
colormap = cm.hot
x = metric['reg']
z = metric['features'] 
y = metric['weight']
c = metric['MRR']

fig = plt.figure()

ax = fig.add_subplot(111, projection='3d')
img = ax.scatter(x, y, z, c=c, cmap=plt.jet())
ax.set_xlabel('reg')
ax.set_zlabel('features')
ax.set_ylabel('weight')
fig.colorbar(img)
plt.show()